In [70]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os 
import glob

pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',100)

In [ ]:
# Map information to get regions from counties

region3_map = {

    "Jasper County": "NC", "Lake County": "NC", "Laporte County": "NC", "Newton County": "NC", "Porter County": "NC", "Pulaski County": "NC", "Starke County": "NC", "White County": "NC",
    "Kosciusko County":"NC","Wabash County":"NC","St Joseph County":"NC","Cass County": "NC", "Fulton County": "NC", "Howard County": "C", "Miami County": "NC", "Tippecanoe County": "NC", "Tipton County": "C", "Carroll County": "NC", "Clinton County": "C","Marshall County":"NC","Elkhart County":"NC",
    "Adams County": "NE", "Allen County": "NE", "Dekalb County": "NE", "Huntington County": "NE", "Lagrange County": "NE", "Noble County": "NE", "Steuben County": "NE", "Wells County": "NE", "Whitley County": "NE",
    "Benton County": "NC", "Fountain County": "NC", "Montgomery County": "NC", "Parke County": "NC", "Putnam County": "NC", "Vermillion County": "NC", "Warren County": "NC","Vigo County": "NC","Clay County": "NC",
    "Boone County": "C", "Hamilton County": "C", "Hancock County": "C", "Hendricks County": "C", "Johnson County": "C", "Madison County": "C", "Marion County": "C", "Morgan County": "C", "Shelby County": "C",
    "Blackford County": "NE","Union County": "NE","Fayette County": "NE", "Delaware County": "NE", "Grant County": "C", "Henry County": "NE", "Jay County": "NE", "Randolph County":"NE","Rush County":"C","Wayne County":"NE",    
    "Daviess County": "NC", "Sullivan County": "NC","Gibson County": "NC", "Knox County": "NC", "Perry County": "NC", "Pike County": "NC", "Posey County": "NC", "Spencer County": "NC", "Vanderburgh County": "NC", "Warrick County": "NC",
    "Brown County": "NC", "Crawford County": "NC", "Dubois County": "NC", "Greene County": "NC", "Lawrence County": "NC", "Martin County": "NC", "Monroe County": "NC", "Orange County": "NC", "Owen County": "NC", "Washington County": "NC",
    "Bartholomew County": "C", "Clark County": "NC", "Decatur County": "C", "Dearborn County": "NC", "Floyd County": "NC", "Franklin County": "NC", "Harrison County": "NC", "Jackson County": "NC", "Jefferson County": "NC", "Jennings County": "NC", "Ohio County": "NC", "Ripley County": "NC", "Scott County": "NC", "Switzerland County": "NC"
}

$$Y = a + b * N - |c| * N^2$$

In [6]:
# Loading both dataframes
gtd1=pd.read_csv("/workspace/GTD1.csv")
gtd2=pd.read_csv("/workspace/GTD2.csv")
gtd1.head(5)

,County,Year,rTotN_kgha,rY15_mtha
0,White,22,248,13.0
1,White,22,228,10.9
2,White,22,229,13.9
3,White,22,229,13.9
4,White,22,231,13.8


#### GTD1 Transformation

In [ ]:
# GTD1 Transformation

new_gtd1=gtd1
# Fixing years
new_gtd1['Year']=new_gtd1['Year']+2000
# Fixing counties
new_gtd1["County"] = new_gtd1["County"].replace({
    "White": "White County",
    "Randolph": "Randolph County",
    "Dubois": "Dubois County",
    "Marshall": "Marshall County",
    "Clay": "Clay County"
})
# Mapping regions
new_gtd1['region']= new_gtd1["County"].map(region3_map)

new_gtd1.columns=['county','year','NKg_Ha','yield_ton','region']

new_gtd1

,county,year,NKg_Ha,yield_ton,region
0,White County,2022,248,13.0,NC
1,White County,2022,228,10.9,NC
2,White County,2022,229,13.9,NC
3,White County,2022,229,13.9,NC
4,White County,2022,231,13.8,NC
...,...,...,...,...,...
1575,Clay County,2022,219,19.2,NC
1576,Clay County,2022,242,18.2,NC
1577,Clay County,2022,234,19.0,NC
1578,Clay County,2022,229,18.2,NC


#### GTD2 Transformation

In [ ]:
# Setting rates of nitrogen
nitro=[0,44.60,89.20,133.80,178.40,223.00,267.60]

# Variables of the new GTD2
yield_ton=[]
NKg_Ha=[]
long=[]
lat=[]
county=[]
region=[]
year=[]

for idx,row in gtd2.iterrows():
    yield_per_place=[0]
    for rate in nitro:
        year.append(row['Year'])
        region.append(row['CRD'])
        county.append(row['County'])
        lat.append(row['Latitude'])
        long.append(row['Longitude'])
        NKg_Ha.append(rate/0.892)
        # Assuming that the formula is in bushes/acre (I think)
        yield_bush=float(row['a'])+float(row['b'])*rate-np.abs(float(row['c']))*(rate**2)
        ton=((yield_bush*60)*1.12085)/1000
        
        if yield_per_place[-1]==0:
            yield_per_place.append(ton)
            yield_ton.append(ton)
            
        elif ton < yield_per_place[-1]:
            yield_ton.append(yield_ton[-1])
            
        else:
            yield_ton.append(ton)
            yield_per_place.append(ton)  
    
        
       
# Creating new GTD2
new_gtd2=pd.DataFrame({
    'yield_ton':yield_ton,
    'NKg_Ha':NKg_Ha,
    'long':long,
    'lat':lat,
    'county':county,
    'region':region,
    'year':year
    })

# Mapping Region

new_gtd2=new_gtd2[['yield_ton','NKg_Ha','county','region','year']]

county_map={
    "Tippecanoe": "Tippecanoe County",
    "Ripley": "Ripley County",
    "Hamilton": "Hamilton County",
    "Gibson": "Gibson County",
    "Vigo": "Vigo County",
    "Grant": "Grant County",
    "Randolph": "Randolph County",
    "Hendricks": "Hendricks County",
    "Lawrence": "Lawrence County",
    "Decatur": "Decatur County",
    "Whitley": "Whitley County",
    "Clay": "Clay County",
    "Henry": "Henry County",
    "Porter": "Porter County",
    "Jennings": "Jennings County",
    "Knox": "Knox County",
    "Benton": "Benton County",
    "Blackford": "Blackford County",
    "Pulaski": "Pulaski County",
    "Clinton": "Clinton County",
    "Lake": "Lake County",
    "Carroll": "Carroll County",
    "Adams": "Adams County",
    "Marshall": "Marshall County",
    "Elkhart": "Elkhart County",
    "Madison": "Madison County",
    "Johnson": "Johnson County",
    "Jasper": "Jasper County",
    "Cass": "Cass County",
    "Vanderburgh": "Vanderburgh County",
    "Shelby": "Shelby County",
    "La Porte": "Laporte County",
    "Miami": "Miami County"
}


valid_counties = list(county_map.keys())

new_gtd2 = new_gtd2[new_gtd2["county"].isin(valid_counties)].copy()

new_gtd2["county"] = new_gtd2["county"].replace(county_map)

new_gtd2["region"] = new_gtd2["county"].map(region3_map)



Merging GTD1 and GTD2

In [6]:
GTD=pd.concat([new_gtd2,new_gtd1],ignore_index=True)
GTD

,yield_ton,NKg_Ha,county,region,year
0,8.951108,0.0,Tippecanoe County,NC,2006
1,11.267043,50.0,Tippecanoe County,NC,2006
2,12.857928,100.0,Tippecanoe County,NC,2006
3,13.723763,150.0,Tippecanoe County,NC,2006
4,13.864548,200.0,Tippecanoe County,NC,2006
...,...,...,...,...,...
2730,19.200000,219.0,Clay County,NC,2022
2731,18.200000,242.0,Clay County,NC,2022
2732,19.000000,234.0,Clay County,NC,2022
2733,18.200000,229.0,Clay County,NC,2022


In [7]:
GTD.groupby(['region']).size()

region
C      154
NC    2164
NE     417
dtype: int64

In [8]:
GTD.to_csv("/workspace/workflow/_6EvaluationNotebooks/GTD.csv")